# monomorphic
A small conversion utility to address libraries that do not support polymorphism for python builtin types

## The situation
You've got some code that uses duck typing or extends a builtin python data type.  As an example, one might have a string wrapper:

While most libraries support polymorphism:

In [23]:
from typing import Any

def nice_library_function(v: Any) -> str:
    if isinstance(v, str):
        return f"STRING: {v}"
    elif isinstance(v, int):
        return f"INT: {v}"
    elif isinstance(v, dict):
        return str({k: nice_library_function(v2) for k, v2 in v.items()})
    elif isinstance(v, list):
        return str([nice_library_function(v2) for v2 in v])
    elif isinstance(v, set):
        return str({nice_library_function(v2) for v2 in v})
    else:
        raise ValueError(f"I don't understand type {type(v)}")

One will still encounters code that is a bit brittle. The example below, for instance,
doesn't recognize polymorphic strings or integers.

In [24]:
from typing import Union

def brittle_library_function(v: Union[int, str]) -> Union[str, int, dict, list, set]:
    v_type = type(v)

    if v_type is str:
        return f"STRING: {v}"
    elif v_type is int:
        return f"INT: {v}"
    elif isinstance(v, dict):
        return str({k: brittle_library_function(v2) for k, v2 in v.items()})
    elif isinstance(v, list):
        return str([brittle_library_function(v2) for v2 in v])
    elif isinstance(v, set):
        return str({brittle_library_function(v2) for v2 in v})
    else:
        raise ValueError(f"I don't understand type {type(v)}")

When one is using a subclass (or other polymorphism) of a python base class, the `brittle_library_function` will fail.  In the example below, we have decorated the string class to produce `Rope` -- a string that has an additional element that represents the number of twines:

In [25]:
from typing import Any, Union

class Rope(str):
    def __init__(self, v: Any) -> None:
        self._ntwines = 1
        super().__init__()

    @property
    def twines(self) -> int:
        return self._ntwines

    @twines.setter
    def twines(self, v: int) -> None:
        self._ntwines = v

If one invokes a well behaved library, things work as one would expect:

In [26]:
print(nice_library_function("Hi there"))
print(nice_library_function(Rope("17 feet")))
print(nice_library_function(["e1", 42, Rope("One yard")]))

STRING: Hi there
STRING: 17 feet
['STRING: e1', 'INT: 42', 'STRING: One yard']


But when one tries the brittle function:

In [27]:
print(brittle_library_function("Hi there"))
try:
    print(brittle_library_function(Rope("17 feet")))
except ValueError as e:
    print(f"The library doesn't like me 😞: \n\t{e}")
try:
    print(brittle_library_function(["e1", 42, Rope("One yard")]))
except ValueError as e:
    print(f"It hates my structures 😢: \n\t{e}")

STRING: Hi there
The library doesn't like me 😞: 
	I don't understand type <class '__main__.Rope'>
It hates my structures 😢: 
	I don't understand type <class '__main__.Rope'>


## `monomorphic` to the rescue!
`monomorphic` creates a copy of a python data structure substituting subclasses of builtin types with the types
with the types themselves

In [28]:
!pip install monomorphic --upgrade -q

In [29]:
from monomorphic import monomorph

print(brittle_library_function(monomorph("Hi there")))
print(brittle_library_function(monomorph(Rope("17 feet"))))
print(nice_library_function(["e1", 42, Rope("One yard")]))


STRING: Hi there
STRING: 17 feet
['STRING: e1', 'INT: 42', 'STRING: One yard']
